In [177]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
import matplotlib.pyplot as plt

import numpy as np
import os
import h5py
from matplotlib import pyplot as plt
from matplotlib import gridspec as gridspec
from sklearn.svm import SVC
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn import metrics
from sklearn.decomposition import PCA
from scipy.stats import uniform, loguniform
datadir = '../data'
figdir  = '../data'

fname_data = os.path.join(datadir, 'gas_data.hdf5')   

# UPDATED DATA TO BE USED:
# datasets in fname_data: 
#  - data_1point (sample all 14 gas properties at t_formation) (212x14)
#  - data_5point (sample gas properties at 5 times between start of simulation and t_formation) (212x5*14)
#  - data_10point (sample at 10 different times) (212x10*14)
#  - data_20_point (212x20*14)
#  - data_30_point
#  - data_50_point
#  - sink_IDs (212x1)
#  - labels (multiplicity shortly after formation time, before dynamical ejections from stellar systems) (212x1)
#  - labels_final (multiplicity at the end of the simulation)

# Return the labels, gas properties (sampled at 1 up to 50 times in simulation), and sink_IDs.
def loadData(fname_data):
    with h5py.File(fname_data, 'r') as f:
        sink_IDs     = f['sink_IDs'][()]      # (212,)
        labels       = f['labels'][()]        # (212,)
        labels_final = f['labels_final'][()]  # (212,)
        data_1point  = f['data_1point'][()]   # (212, 1x14)
        data_5point  = f['data_5point'][()]   # (212, 5x14)
        data_10point = f['data_10point'][()]  # (212, 10x14)
        data_20point = f['data_20point'][()]  # (212, 20x14)
        data_30point = f['data_30point'][()]  # (212, 30x14)
        data_50point = f['data_50point'][()]  # (212, 50x14)
        
        data = [data_1point, data_5point, data_10point, data_20point, data_30point, data_50point]
        
        # Replace NaN with 0
        for dset in data:
            dset[np.isnan(dset)] = 0.0
            
        # Replace inf with 0 for now.
        for dset in data:
            dset[np.isinf(dset)] = 0.0
            
        n_sinks = len(sink_IDs)
        
    labels       = np.reshape(labels, (n_sinks, 1))
    labels_final = np.reshape(labels_final, (n_sinks, 1))
        
    return labels, labels_final, data, sink_IDs

# Add option to replace labels ['2', '3', '4'] with just '2' to compare single vs. multiple systems.
def getData(fname_data, num_samples='20', use_labels='initial', relabel_multiplicity=False):
    '''
    num_samples = '1', '5', '10', '20', '30', '50'
    use_labels = 'initial' to use labels, 'final' to use labels_final
    '''
    labels_initial, labels_final, data, sink_IDs = loadData(fname_data)
    
    # Decide which multiplicity labels to use (at formation vs. end of simulation).
    if use_labels == 'initial':
        labels = labels_initial
    else:
        labels = labels_final
    
    # Replace higher-order multiplicity labels with '2'.
    if relabel_multiplicity:
        labels[(labels >= 2)] = 2
        
    y = labels
    if num_samples == '1':
        x = data[0]
    elif num_samples =='5':
        x = data[1]
    elif num_samples == '10':
        x = data[2]
    elif num_samples == '20':
        x = data[3]
    elif num_samples == '30':
        x = data[4]
    else:
        x = data[5]
    return x, y

def preprocessData(x, y):
    #pt   = PowerTransformer(method= 'yeo-johnson', standardize= True)
    scaler = StandardScaler()
    #_trans = pt.fit_transform(x)
    x_trans = scaler.fit_transform(x)
    x_train, x_test, y_train, y_test = train_test_split(x_trans, y, train_size= 0.7, random_state=80)
    return x_train, x_test, np.ravel(y_train), np.ravel(y_test)







In [180]:
x1, y1_2class = getData(fname_data, num_samples='1', use_labels='final', relabel_multiplicity=True) # labels for 2 classes
x30, y30_2class = getData(fname_data, num_samples='30', use_labels='final', relabel_multiplicity=True)
#_, y1_4class = getData(fname_data, num_samples='1', use_labels = 'final', relabel_multiplicity=False) # labels for 4 classes
#_, y30_4class = getData(fname_data, num_samples='30', use_labels='final', relabel_multiplicity=False)

x1_train, x1_test, y1_2class_train, y1_2class_test = preprocessData(x1, y1_2class)
x30_train, x30_test, y30_2class_train, y30_2class_test = preprocessData(x30, y30_2class)

y1_2class_train = y1_2class_train -1
y1_2class_test = y1_2class_test -1
y30_2class_train = y30_2class_train - 1
y30_2class_test = y30_2class_test - 1

In [215]:
model = Sequential([
    layers.Dense(10, activation=layers.LeakyReLU(alpha = 0.01), kernel_regularizer = regularizers.l2(l = 0.01)),
    layers.Dropout(0.2),
    layers.Dense(10, activation=layers.LeakyReLU(alpha = 0.01), kernel_regularizer = regularizers.l2(l = 0.01)),
    layers.Dropout(0.2),
    layers.Dense(10, activation=layers.LeakyReLU(alpha = 0.01), kernel_regularizer = regularizers.l2(l = 0.01)),
    layers.Dropout(0.2),
    layers.Dense(10, activation=layers.LeakyReLU(alpha = 0.01), kernel_regularizer = regularizers.l2(l = 0.01)),
    layers.Dropout(0.2),
    layers.Dense(5, activation = layers.LeakyReLU(alpha = 0.01)),
    layers.Dense(1, activation='sigmoid')
])

adam = Adam(learning_rate = 0.01)

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

model.fit(x1_train, y1_2class_train, epochs = 500, batch_size = 20)

Epoch 1/500
8/8 [==============================] - 0s 629us/step - loss: 1.0129 - accuracy: 0.6622
Epoch 2/500
8/8 [==============================] - 0s 542us/step - loss: 0.8310 - accuracy: 0.7365
Epoch 3/500
8/8 [==============================] - 0s 551us/step - loss: 0.7466 - accuracy: 0.7365
Epoch 4/500
8/8 [==============================] - 0s 482us/step - loss: 0.6536 - accuracy: 0.7365
Epoch 5/500
8/8 [==============================] - 0s 517us/step - loss: 0.6190 - accuracy: 0.7365
Epoch 6/500
8/8 [==============================] - 0s 486us/step - loss: 0.6004 - accuracy: 0.7365
Epoch 7/500
8/8 [==============================] - 0s 545us/step - loss: 0.5830 - accuracy: 0.7365
Epoch 8/500
8/8 [==============================] - 0s 459us/step - loss: 0.5844 - accuracy: 0.7365
Epoch 9/500
8/8 [==============================] - 0s 477us/step - loss: 0.5775 - accuracy: 0.7365
Epoch 10/500
8/8 [==============================] - 0s 474us/step - loss: 0.5717 - accuracy: 0.7365
Epoch 11/

In [216]:
model.evaluate(x1_test, y1_2class_test)

2/2 [==============================] - 0s 1ms/step - loss: 0.7120 - accuracy: 0.6562


[0.7119860649108887, 0.65625]